# Overview

This notebook is used for exploring the data and documenting findings.

# Initial setup

In [1]:
# Import libraries
import glob
import pandas as pd
import numpy as np
import sqlite3
from sklearn.model_selection import train_test_split

In [2]:
# Load raw data
file_path = "../data/raw"
raw_files = glob.glob(file_path + "/*.csv")

file_list = []

for file in raw_files:
    file_list.append(pd.read_csv(file, index_col=None, header=0))

raw_data = pd.concat(file_list, axis=0, ignore_index=True)

In [3]:
print(raw_data.shape)
raw_data.head()

(3818852, 5)


,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation
0,52156190,6/27/2015 12:47:00,Donation,Whole Blood,Center
1,52825057,2/26/2015 9:53:00,Donation,2 Units RBC,Mobile
2,53025596,9/8/2015 16:49:59,Donation,Whole Blood,Mobile
3,2056692,8/26/2015 12:15:00,Donation,Whole Blood,Mobile
4,52879521,1/26/2015 17:18:00,Incomplete,Whole Blood,Center


# Clean raw data

In [4]:
# Show where we have NaNs/nulls
raw_data[raw_data.isnull().any(axis=1)]

,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation
9589,53639912,6/23/2015 14:20:59,Donation,Whole Blood,NaN
30664,52877130,4/13/2015 16:49:00,Donation,Whole Blood,NaN
55262,52869439,9/11/2015 13:38:00,Donation,Whole Blood,NaN
236526,53594657,6/23/2015 15:55:00,Donation,Whole Blood,NaN
240689,53165336,8/7/2015 17:19:59,Donation,Whole Blood,NaN
268039,53256162,11,NaN,NaN,NaN
284335,53373866,7/11/2016 17:42,Donation,Whole Blood,NaN
596076,52879414,9/20/2016 14:42,Donation,Whole Blood,NaN
670560,52971104,12/29/2016 13:49,Donation,Whole Blood,NaN
737587,53591355,3/23/2016 14:57,Donation,2 Units RBC,NaN


In [5]:
# Drop rows with nulls
cleaned_data = raw_data.dropna(axis=0, how='any')

In [6]:
# Clean up RegistrationTime: pad time with '00' seconds if only HH:MM is shown, then convert to a datetime type
cleaned_data['RegistrationTime'] = cleaned_data['RegistrationTime'].apply(lambda c: c + ':00' if c.count(':') < 2 else c)
cleaned_data['RegistrationTime'] = pd.to_datetime(cleaned_data['RegistrationTime'], format='%m/%d/%Y %H:%M:%S')
cleaned_data['RegistrationTime'].head()

0   2015-06-27 12:47:00
1   2015-02-26 09:53:00
2   2015-09-08 16:49:59
3   2015-08-26 12:15:00
4   2015-01-26 17:18:00
Name: RegistrationTime, dtype: datetime64[ns]

In [7]:
cleaned_data.head()

,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation
0,52156190,2015-06-27 12:47:00,Donation,Whole Blood,Center
1,52825057,2015-02-26 09:53:00,Donation,2 Units RBC,Mobile
2,53025596,2015-09-08 16:49:59,Donation,Whole Blood,Mobile
3,2056692,2015-08-26 12:15:00,Donation,Whole Blood,Mobile
4,52879521,2015-01-26 17:18:00,Incomplete,Whole Blood,Center


# Test pure Pandas

In [20]:
subset_data = cleaned_data[cleaned_data['Random_ID'].isin([52437236, 54374547])]

In [21]:
subset_data

,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation
12656,52437236,2015-04-14 12:31:00,Donation,Platelets and Concurrent Plasma,Center
15897,52437236,2015-03-13 12:50:59,Donation,Platelets and Concurrent Plasma,Center
60492,52437236,2015-10-30 12:18:00,Donation,Platelet Apheresis,Center
71652,52437236,2015-12-16 10:35:59,Incomplete,Platelet Apheresis,Center
74700,52437236,2015-12-22 10:04:59,Donation,Platelet Apheresis,Center
...,...,...,...,...,...
3406300,54374547,2019-04-23 16:41:00,Donation,Whole Blood,Center
3679322,54374547,2019-06-22 15:41:00,Donation,Whole Blood,Mobile
3716121,52437236,2019-07-02 12:21:59,Donation,Platelet Apheresis,Center
3746497,52437236,2019-07-20 13:13:00,Donation,Platelet Apheresis,Center


In [22]:
cutoff_dates = ['2016-02-29 23:59:59', '2016-05-31 23:59:59']

In [23]:
# for cutoff_date in cutoff_dates:
cutoff = pd.to_datetime(cutoff_dates[0], format='%Y-%m-%d %H:%M:%S')
target_start_date = cutoff + pd.Timedelta(seconds=1)
target_end_date = cutoff + pd.Timedelta(days=30)
print(f"Cutoff date: {cutoff}")
print(f"Target period: {target_start_date} - {target_end_date}")

Cutoff date: 2016-02-29 23:59:59
Target period: 2016-03-01 00:00:00 - 2016-03-30 23:59:59


In [24]:
# Filter records to those that are eligible for the current cutoff date
cutoff_history = subset_data[subset_data['RegistrationTime'] <= cutoff]
# Add CutoffDate as a literal value so we can combine all cutoff date histories
cutoff_history['CutoffDate'] = cutoff
cutoff_history

,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation,CutoffDate
12656,52437236,2015-04-14 12:31:00,Donation,Platelets and Concurrent Plasma,Center,2016-02-29 23:59:59
15897,52437236,2015-03-13 12:50:59,Donation,Platelets and Concurrent Plasma,Center,2016-02-29 23:59:59
60492,52437236,2015-10-30 12:18:00,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59
71652,52437236,2015-12-16 10:35:59,Incomplete,Platelet Apheresis,Center,2016-02-29 23:59:59
74700,52437236,2015-12-22 10:04:59,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59
153157,54374547,2015-02-26 15:35:00,Donation,Whole Blood,Center,2016-02-29 23:59:59
184026,52437236,2015-04-23 09:52:00,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59
324709,52437236,2016-02-04 11:53:00,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59
571062,52437236,2016-02-20 13:22:00,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59
639975,52437236,2016-01-21 11:36:00,Donation,Platelet Apheresis,Center,2016-02-29 23:59:59


In [35]:
cutoff

Timestamp('2016-02-29 23:59:59')

In [28]:
# Calculate recency: difference between most recent donation date per donor, and the current cutoff date
# Calculate time: total days since first registration
recency = cutoff_history.groupby(by='Random_ID', as_index=False).agg({'RegistrationTime': ['min', 'max']})
recency.columns = recency.columns.droplevel(0)
recency.columns = ['Random_ID', 'FirstRegistrationTime', 'LastRegistrationTime']
recency['DaysSinceLastRegistration'] = (cutoff - recency['LastRegistrationTime']).dt.days  # Just capture the days portion
recency['DaysSinceFirstRegistration'] = (cutoff - recency['FirstRegistrationTime']).dt.days
recency

,Random_ID,FirstRegistrationTime,LastRegistrationTime,DaysSinceLastRegistration,DaysSinceFirstRegistration
0,52437236,2015-03-13 12:50:59,2016-02-20 13:22:00,9,353
1,54374547,2015-02-26 15:35:00,2015-02-26 15:35:00,368,368


In [29]:
# Calculate frequency: number of registrations for donation in the history period
frequency = cutoff_history.groupby(by='Random_ID', as_index=False).agg({'RegistrationTime': 'count'}).rename(columns={'RegistrationTime': 'PastRegistrations'})
frequency

,Random_ID,TotalRegistrations
0,52437236,10
1,54374547,1


In [74]:
# Calculate frequency by DonationType
frequency_type = cutoff_history.groupby(by=['Random_ID', 'DonationType'], as_index=False) \
                               .agg({'RegistrationTime': 'count'}).rename(columns={'RegistrationTime': 'TotalRegistrations'}) \
                               .pivot(index='Random_ID', columns='DonationType', values='TotalRegistrations') \
                               .reset_index().rename_axis(None, axis=1)  # Make Random_ID a column; remove index name
frequency_type.columns = ['Random_ID'] + ['Past' + col_name.replace(' ', '') + 'Registrations' for col_name in frequency_type.columns if col_name != 'Random_ID']

In [75]:
frequency_type

,Random_ID,PlateletApheresisRegistrations,PlateletsandConcurrentPlasmaRegistrations,WholeBloodRegistrations
0,52437236,8.0,2.0,NaN
1,54374547,NaN,NaN,1.0


In [30]:
# Calculate monetary: total volume of donation in the history period?

In [31]:
# Filter data down to target period
cutoff_target = subset_data[(subset_data['RegistrationTime'] >= target_start_date) & (subset_data['RegistrationTime'] <= target_end_date)]
cutoff_target

,Random_ID,RegistrationTime,OutCome,DonationType,DonationLocation
827760,52437236,2016-03-22 12:24:00,Donation,Platelets and Concurrent Plasma,Center


In [76]:
# Calculate base measure for target: how many total registrations each donor had in the target period
response = cutoff_target.groupby(by='Random_ID', as_index=False).agg({'RegistrationTime': 'count'}).rename(columns={'RegistrationTime': 'TargetRegistrations'})
response

,Random_ID,TargetRegistrations
0,52437236,1


In [77]:
# Calculate sub-targets: how many registrations of each DonationType each donor had in the target period
response_type = cutoff_target.groupby(by=['Random_ID', 'DonationType'], as_index=False) \
                             .agg({'RegistrationTime': 'count'}).rename(columns={'RegistrationTime': 'TotalRegistrations'}) \
                             .pivot(index='Random_ID', columns='DonationType', values='TotalRegistrations') \
                             .reset_index().rename_axis(None, axis=1)  # Make Random_ID a column; remove index name
response_type.columns = ['Random_ID'] + ['Target' + col_name.replace(' ', '') + 'Registrations' for col_name in response_type.columns if col_name != 'Random_ID']

In [78]:
response_type

,Random_ID,TargetPlateletsandConcurrentPlasmaRegistrations
0,52437236,1


In [34]:
# Combine datasets
data = pd.merge(recency, frequency, how='left').merge(response, how='left')
data['RegisteredInTargetPeriod'] = data['Registrations'].apply(lambda x: 0 if pd.isna(x) else 1)
data['CutoffDate'] = cutoff
data['TargetPeriodStartDate'] = target_start_date
data['TargetPeriodEndDate'] = target_end_date
data

,Random_ID,FirstRegistrationTime,LastRegistrationTime,DaysSinceLastRegistration,DaysSinceFirstRegistration,TotalRegistrations,Registrations,RegisteredInTargetPeriod,CutoffDate,TargetPeriodStartDate,TargetPeriodEndDate
0,52437236,2015-03-13 12:50:59,2016-02-20 13:22:00,9,353,10,1.0,1,2016-02-29 23:59:59,2016-03-01,2016-03-30 23:59:59
1,54374547,2015-02-26 15:35:00,2015-02-26 15:35:00,368,368,1,NaN,0,2016-02-29 23:59:59,2016-03-01,2016-03-30 23:59:59
